<a href="https://colab.research.google.com/github/NicoleRichards1998/FinRL/blob/master/DayTrading_FinRL_JSE_Singe_Stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Disclaimer: Nothing herein is financial advice, and NOT a recommendation to trade real money. Many platforms exist for simulated trading (paper trading) which can be used for building and developing the methods discussed. Please use common sense and always first consult a professional before trading or investing.

# Part 1: Install FinRL

In [1]:
## install finrl library
!pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git

  Cloning https://github.com/AI4Finance-LLC/FinRL-Library.git to /tmp/pip-req-build-y8037p9t
  Running command git clone -q https://github.com/AI4Finance-LLC/FinRL-Library.git /tmp/pip-req-build-y8037p9t
  Cloning https://github.com/quantopian/pyfolio.git to /tmp/pip-install-trav9v8j/pyfolio_82fb1fc64e114230bb04b1ca2694c6ac
  Running command git clone -q https://github.com/quantopian/pyfolio.git /tmp/pip-install-trav9v8j/pyfolio_82fb1fc64e114230bb04b1ca2694c6ac
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-trav9v8j/elegantrl_0570ed731bcd4912be7dfd3060c4e976
  Running command git clone -q https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-trav9v8j/elegantrl_0570ed731bcd4912be7dfd3060c4e976
     |████████████████████████████████| 234 kB 8.2 MB/s 
     |████████████████████████████████| 177 kB 44.9 MB/s 
     |████████████████████████████████| 53.2 MB 9.9 MB/s 
     |████████████████████████████████| 1.2 MB 38.4 MB/s 
     |█████

In [2]:
!pip install tensorboardX

In [3]:
!pip uninstall -y pyarrow
!pip uninstall -y ray # clean removal of previous install, otherwise version number may cause pip not to upgrade

Found existing installation: pyarrow 6.0.1
Uninstalling pyarrow-6.0.1:
  Successfully uninstalled pyarrow-6.0.1
Found existing installation: ray 1.12.0
Uninstalling ray-1.12.0:
  Successfully uninstalled ray-1.12.0


In [4]:
!python -m pip install --upgrade pip

     |████████████████████████████████| 2.1 MB 5.2 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


In [5]:
!pip install tf-estimator-nightly==2.8.0.dev2021122109
!pip install -U https://s3-us-west-2.amazonaws.com/ray-wheels/latest/ray-2.0.0.dev0-cp37-cp37m-manylinux2014_x86_64.whl # minimal install
!pip install lz4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 KB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 MB 10.7 MB/s eta 0:00:00


In [1]:
!pip install git+https://github.com/NicoleRichards1998/FinRL.git

  Cloning https://github.com/NicoleRichards1998/FinRL.git to /tmp/pip-req-build-nb08wihv
  Running command git clone -q https://github.com/NicoleRichards1998/FinRL.git /tmp/pip-req-build-nb08wihv
  Cloning https://github.com/quantopian/pyfolio.git to /tmp/pip-install-m8_2sdqu/pyfolio_a7500bf8685e468a8cb850209b40fb2d
  Running command git clone -q https://github.com/quantopian/pyfolio.git /tmp/pip-install-m8_2sdqu/pyfolio_a7500bf8685e468a8cb850209b40fb2d
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-m8_2sdqu/elegantrl_fa72913af0f147c491843c9140dad887
  Running command git clone -q https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-m8_2sdqu/elegantrl_fa72913af0f147c491843c9140dad887
     |████████████████████████████████| 234 kB 7.9 MB/s 
     |████████████████████████████████| 177 kB 36.8 MB/s 
     |████████████████████████████████| 53.2 MB 189 kB/s 
     |████████████████████████████████| 1.2 MB 44.9 MB/s 
     |█████████████

In [ ]:
print("Setting up colab environment")
!pip uninstall -y -q pyarrow
!pip install -q -U ray[tune]
!pip install -q ray[debug]

# A hack to force the runtime to restart, needed to include the above dependencies.
print("Done installing! Restarting via forced crash (this is not an issue).")
import os
os._exit(0)

Setting up colab environment


## Import related modules

In [1]:
from finrl.finrl_meta.env_stock_trading.env_stocktrading_np import DayTradingEnv as StockTradingEnv1
from finrl.finrl_meta.env_stock_trading.env_stocktrading_np import StockTradingEnv
from finrl.finrl_meta.env_stock_trading.env_stocktrading_np import ChangedStockTradingEnv as StockTradingEnv2
from finrl.finrl_meta.env_stock_trading.env_stock_papertrading import AlpacaPaperTrading
from finrl.finrl_meta.data_processor import DataProcessor
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from finrl.agents.rllib.models import DRLAgent as DRLAgent_rllib

import numpy as np
import pandas as pd
import ray

from ray.rllib.agents.a3c import a2c
from ray.rllib.agents.ddpg import ddpg, td3
from ray.rllib.agents.ppo import ppo
from ray.rllib.agents.ppo.ppo import PPOTrainer
from ray.rllib.agents.sac import sac
from ray.tune.logger import (
    CSVLoggerCallback,
    JsonLoggerCallback,
    JsonLogger,
    CSVLogger,
    TBXLoggerCallback,
    TBXLogger,
)
from ray.tune.logger import pretty_print

from google.colab import files
from datetime import date, timedelta, datetime

from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.core.arrays import string_

import pytz
import exchange_calendars as tc

from stockstats import StockDataFrame as Sdf

import psutil
import ray
ray._private.utils.get_system_memory = lambda: psutil.virtual_memory().total

/usr/local/lib/python3.7/dist-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


## Set variables

In [2]:
model_algorithm = 'ppo'
starting_capital = 1e6

Number_Train_Days = 10

Number_Test_Days = 10

In [3]:
JSEIndexes = [ 'ACL' ]

In [3]:
JSEIndexes = [ 
  'SNH',
  'WHL',
 'FSR',
 'RMH',
 'BLU',
 'SSW',
 'LAB',
 'TGA',
 'SPG',
 'PPC',
 'OMU',
 'GFI',
 'NPK',
 'HAR',
 'LHC',
 'TSG',
 'ACL',
 'PPH',
 'SBK',
 'PPE',
 'CML',
 'MRF',
 'MTN',
 'AVI',
 'SOL',
 'EXX',
 'TRU',
 'NED',
 'TCP',
 'WSL'
 ]

In [4]:
ticker_list = JSEIndexes
action_dim = len(ticker_list)

INDICATORS = ['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'dx_30', 'close_30_sma', 'close_60_sma']
tech_indicator_list = INDICATORS
env = StockTradingEnv2

# amount + (turbulence, turbulence_bool) + (price, shares, cd (holding time)) * stock_dim + tech_dim
state_dim = 1 + 2 + 3 * action_dim + len(INDICATORS) * action_dim

episodes_per_day = 100
tr_batch_size = episodes_per_day*480

rllib_params = {"lr": 5e-5, "train_batch_size": tr_batch_size, "gamma": 0.99}

MODELS = {"a2c": a2c, "ddpg": ddpg, "td3": td3, "sac": sac, "ppo": ppo}

In [21]:
action_dim

1

# Part 2: Train the agent

## Train

In [5]:
%load_ext tensorboard

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd /content/drive/MyDrive/csvfiles/

/content/drive/MyDrive/csvfiles


In [7]:
data_df = pd.read_csv("JSE_Minute_Data_With_Turbulance_Indicator.csv")

In [8]:
df = data_df.copy()
df = df.sort_values(by=["tic", "date"])
x = 9620
dic_tickers = {}

for i in range(30):

  dic_tickers[i] = df.iloc[(i*x):((i+1)*x),]

In [10]:
dic_tickers[0]

,date,open,high,low,close,volume,tic,macd,boll_ub,boll_lb,rsi_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2022-02-28 09:00:00,845,849,845,849,1395,ACL,0.000000,0.000000,0.000000,0.000000,0.000000,849.000000,849.000000,0.000000
30,2022-02-28 09:01:00,845,845,845,845,6,ACL,-0.089744,852.656854,841.343146,0.000000,0.000000,847.000000,847.000000,0.000000
60,2022-02-28 09:02:00,845,845,845,845,0,ACL,-0.114351,850.952135,841.714531,0.000000,0.000000,846.333333,846.333333,0.000000
90,2022-02-28 09:03:00,845,845,845,845,0,ACL,-0.122741,850.000000,842.000000,0.000000,0.000000,846.000000,846.000000,0.000000
120,2022-02-28 09:04:00,845,845,845,845,0,ACL,-0.124715,849.377709,842.222291,0.000000,0.000000,845.800000,845.800000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288450,2022-03-25 16:56:00,916,916,916,916,0,ACL,0.461104,917.842974,911.857026,51.922110,3.543063,914.400000,915.083333,0.058522
288480,2022-03-25 16:57:00,916,916,916,916,0,ACL,0.472535,917.842974,911.857026,51.922110,3.543063,914.433333,915.050000,0.058334
288510,2022-03-25 16:58:00,916,916,916,916,0,ACL,0.476106,917.842974,911.857026,51.922110,3.543063,914.466667,915.016667,0.060164
288540,2022-03-25 16:59:00,916,916,916,916,0,ACL,0.473478,917.842974,911.857026,51.922110,3.543063,914.500000,914.983333,0.057776


In [9]:
trading_days = pd.date_range(start = '02/28/22', end = '03/25/22')

trading_days_List = trading_days[0:].strftime('%m/%d/%y').to_numpy().tolist()
count = 0
delete_number = 0
while count < 26:
    if (trading_days[count].weekday()>=5):
      del trading_days_List[delete_number]
      delete_number = delete_number - 1
    count = count + 1
    delete_number = delete_number + 1 

In [10]:
x = 481
df = dic_tickers[0].copy()
dic = {}

for i in range(20):

  dic[i] = df.iloc[(i*x):((i+1)*x),]
  dic[i] = dic[i].sort_values(by=["tic", "date"])
  dic[i] = dic[i].reset_index(drop=True)
  for j in range(len(dic[i])):
    dic[i]['date'][j] = datetime.strptime(dic[i]['date'][j], '%Y-%m-%d %H:%M:%S')
  dic[i] = dic[i].sort_values(by=["date"])


In [14]:
dic[0]

,date,open,high,low,close,volume,tic,macd,boll_ub,boll_lb,rsi_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2022-02-28 09:00:00,845,849,845,849,1395,ACL,0.000000,0.000000,0.000000,0.000000,0.000000,849.000000,849.000000,0.000000
1,2022-02-28 09:01:00,845,845,845,845,6,ACL,-0.089744,852.656854,841.343146,0.000000,0.000000,847.000000,847.000000,0.000000
2,2022-02-28 09:02:00,845,845,845,845,0,ACL,-0.114351,850.952135,841.714531,0.000000,0.000000,846.333333,846.333333,0.000000
3,2022-02-28 09:03:00,845,845,845,845,0,ACL,-0.122741,850.000000,842.000000,0.000000,0.000000,846.000000,846.000000,0.000000
4,2022-02-28 09:04:00,845,845,845,845,0,ACL,-0.124715,849.377709,842.222291,0.000000,0.000000,845.800000,845.800000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476,2022-02-28 16:56:00,885,885,885,885,0,ACL,1.569266,889.469544,871.030456,58.192359,11.962376,880.333333,881.450000,0.064415
477,2022-02-28 16:57:00,885,885,885,885,0,ACL,1.582383,889.854655,871.645345,58.192359,11.962376,880.500000,881.550000,0.064770
478,2022-02-28 16:58:00,885,885,885,885,0,ACL,1.574628,890.120412,872.379588,58.192359,11.962376,880.666667,881.683333,0.063195
479,2022-02-28 16:59:00,885,885,885,885,0,ACL,1.550607,890.256963,873.243037,58.192359,11.962376,880.800000,881.733333,185.490985


In [12]:
x = 14430
df = data_df.copy()
dic = {}

for i in range(20):

  dic[i] = df.iloc[(i*x):((i+1)*x),]


In [11]:
def df_to_array(df, if_vix):
        df = df.copy()
        unique_ticker = df.tic.unique()
        if_first_time = True
        for tic in unique_ticker:
            if if_first_time:
                price_array = df[df.tic == tic][["close"]].values
                tech_array = df[df.tic == tic][tech_indicator_list].values
                if if_vix:
                    turbulence_array = df[df.tic == tic]["VIXY"].values
                else:
                    turbulence_array = df[df.tic == tic]["turbulence"].values
                if_first_time = False
            else:
                price_array = np.hstack(
                    [price_array, df[df.tic == tic][["close"]].values]
                )
                tech_array = np.hstack(
                    [tech_array, df[df.tic == tic][tech_indicator_list].values]
                )
        print("Successfully transformed into array")
        return price_array, tech_array, turbulence_array

In [12]:
def get_config_Train(dic, Number_Days):  

    env_config = []

    for index in range(Number_Days):

      print(index)

      price_array, tech_array, turbulence_array = df_to_array(dic[index], False)

      config = {
          "price_array": price_array,
          "tech_array": tech_array,
          "turbulence_array": turbulence_array,
          "if_train": True,
      }

      env_config.append(config)

    return env_config

In [13]:
train_env_config = get_config_Train(dic, Number_Train_Days)

0
Successfully transformed into array
1
Successfully transformed into array
2
Successfully transformed into array
3
Successfully transformed into array
4
Successfully transformed into array


In [14]:
def training(
        env_config,
        drl_lib,
        env,
        model_name,
        init_ray=True,
        **kwargs
):
   
    env_instance = env(config=env_config)

    print(env_instance)

    rllib_params = kwargs.get("rllib_params")
    
    agent_rllib = DRLAgent_rllib(
            env=env,
            price_array=env_config['price_array'],
            tech_array=env_config['tech_array'],
            turbulence_array=env_config['turbulence_array'],
        )
    
    print(agent_rllib)

    model, model_config = agent_rllib.get_model(model_name)

    print('got model_config')

    model_config["lr"] = rllib_params["lr"]
    model_config["train_batch_size"] = rllib_params["train_batch_size"]
    model_config["gamma"] = rllib_params["gamma"]

    #model_config["model"]["use_lstm"] = True
    #model_config["model"]["lstm_cell_size"] = 256
    #model_config["model"]["lstm_use_prev_action"] = True
    #model_config["model"]["lstm_use_prev_reward"] = True

    if model_name not in MODELS:
            raise NotImplementedError("NotImplementedError")
    if init_ray:
            ray.init(
                ignore_reinit_error=True
            )  # Other Ray APIs will not work until `ray.init()` is called.
    print('ray initialised')

    if model_name == "ppo":
            trainer = model.PPOTrainer(env=env, config=model_config)
    elif model_name == "a2c":
            trainer = model.A2CTrainer(env=env, config=model_config)
    elif model_name == "ddpg":
            trainer = model.DDPGTrainer(env=env, config=model_config)
    elif model_name == "td3":
            trainer = model.TD3Trainer(env=env, config=model_config)
    elif model_name == "sac":
            trainer = model.SACTrainer(env=env, config=model_config)

    print('got trainer')
    return(trainer)

In [15]:
def train_model(
        total_episodes,
        env_config_list,
        drl_lib,
        env,
        model_name,
        init_ray=True,
        **kwargs
):

        s = "{:3d} reward {:6.2f}/{:6.2f}/{:6.2f} len {:6.2f} saved {}"

        trainer = training(
                        env_config = env_config_list[0],
                        drl_lib='rllib', 
                        env=env,
                        model_name = model_algorithm, 
                        init_ray=True,
                        rllib_params = rllib_params,
                        local_dir="./trained_models",#Saving tensorboard plots
                        reuse_actors=True,
                        callbacks=[TBXLoggerCallback()]
                        #callbacks = {#"on_episode_start": on_episode_start,
                                    #"on_episode_step": on_episode_step,
                                    #"on_episode_end": on_episode_end,
                                    #"on_sample_end": on_sample_end,
                                    #"on_postprocess_traj": on_postprocess_traj,
                                    #"on_train_result": on_train_result,
                                    #},
                                    )
        result = trainer.train()
        print(pretty_print(result))
            
        file_name = trainer.save("/tmp/rllib_checkpoint")

        print(s.format(
              1,
              result["episode_reward_min"],
              result["episode_reward_mean"],
              result["episode_reward_max"],
              result["episode_len_mean"],
              file_name
            ))

        ray.shutdown()

        for episode in range(total_episodes-1):
            trainer = training(
                        env_config = env_config_list[episode+1],
                        drl_lib='rllib',
                        env=env,
                        model_name = model_algorithm, 
                        init_ray=True,
                        rllib_params = rllib_params,
                        local_dir="./trained_models",#Saving tensorboard plots
                        reuse_actors=True,
                        callbacks=[TBXLoggerCallback()])
                        
            trainer.restore(file_name)
            result = trainer.train()
            print(pretty_print(result))
            
            file_name = trainer.save("/tmp/rllib_checkpoint")

            print(s.format(
              episode + 2,
              result["episode_reward_min"],
              result["episode_reward_mean"],
              result["episode_reward_max"],
              result["episode_len_mean"],
              file_name
            ))

            ray.shutdown()
  
        return file_name

In [16]:
ray.shutdown()

In [17]:
checkpoint_path = train_model(
      total_episodes = len(train_env_config),
      env_config_list = train_env_config,
      drl_lib='rllib', 
      env=env,
      model_name = model_algorithm, 
      rllib_params = rllib_params)

<StockTradingEnv instance>
got model_config


2022-05-03 12:18:39,885	INFO services.py:1462 -- View the Ray dashboard at http://127.0.0.1:8265
2022-05-03 12:18:42,822	INFO trainer.py:2296 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2022-05-03 12:18:42,832	INFO ppo.py:269 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-05-03 12:18:42,833	INFO trainer.py:867 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


ray initialised


(RolloutWorker pid=911) 2022-05-03 12:18:52,364	WARNING rollout_worker.py:499 -- We've added a module for checking environments that are used in experiments. It will cause your environment to fail if your environment is not set upcorrectly. You can disable check env by setting `disable_env_checking` to True in your experiment config dictionary. You can run the environment checking module standalone by calling ray.rllib.utils.check_env(env).
(RolloutWorker pid=911) 2022-05-03 12:18:52,364	WARNING env.py:121 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
(RolloutWorker pid=910) 2022-05-03 12:18:52,319	WARNING rollout_worker.py:499 -- We've added a module for checking environments that are used in experiments. It will cause your environment to fail if your environment is not set upcorrectly.

got trainer


...

ray initialised


(RolloutWorker pid=4794) 2022-05-03 12:29:51,814	WARNING rollout_worker.py:499 -- We've added a module for checking environments that are used in experiments. It will cause your environment to fail if your environment is not set upcorrectly. You can disable check env by setting `disable_env_checking` to True in your experiment config dictionary. You can run the environment checking module standalone by calling ray.rllib.utils.check_env(env).
(RolloutWorker pid=4794) 2022-05-03 12:29:51,814	WARNING env.py:121 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
(RolloutWorker pid=4795) 2022-05-03 12:29:51,995	WARNING rollout_worker.py:499 -- We've added a module for checking environments that are used in experiments. It will cause your environment to fail if your environment is not set upcorrect

got trainer


2022-05-03 12:29:56,953	INFO trainable.py:535 -- Restored on 172.28.0.2 from checkpoint: /tmp/rllib_checkpoint/checkpoint_000004/checkpoint-4
2022-05-03 12:29:56,963	INFO trainable.py:543 -- Current state after restoring: {'_iteration': 4, '_timesteps_total': 144000, '_time_total': 547.1378355026245, '_episodes_total': 296}


agent_timesteps_total: 180000
custom_metrics: {}
date: 2022-05-03_12-32-53
done: false
episode_len_mean: 480.0
episode_media: {}
episode_reward_max: -4.254409511080286
episode_reward_mean: -8.19877687764129
episode_reward_min: -13.582756520703583
episodes_this_iter: 74
episodes_total: 370
experiment_id: 98653530c34a49ad90c187aceefb027c
hostname: 0271d767eda1
info:
  learner:
    default_policy:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 0.10000000149011612
        cur_lr: 4.999999873689376e-05
        entropy: 1.3287290334701538
        entropy_coeff: 0.0
        kl: 0.00861992035061121
        model: {}
        policy_loss: -0.004803549498319626
        total_loss: 0.2904202342033386
        vf_explained_var: -0.009767014533281326
        vf_loss: 0.29436179995536804
      num_agent_steps_trained: 128.0
  num_agent_steps_sampled: 180000
  num_agent_steps_trained: 180000
  num_steps_sampled: 180000
  num_steps_trained: 180000
  num_steps_trained_this_iter: 3600

In [ ]:
print(checkpoint_path)

/tmp/rllib_checkpoint/checkpoint_000001/checkpoint-1


In [ ]:
%tensorboard --logdir=~/ray_results

## Test

In [ ]:
def get_config_Test(dic, Number_Days):  

    env_config = []

    for index in range(Number_Days):

      print(index)

      price_array, tech_array, turbulence_array = df_to_array(dic[index+Number_Train_Days], False)

      config = {
          "price_array": price_array,
          "tech_array": tech_array,
          "turbulence_array": turbulence_array,
          "if_train": True,
      }

      env_config.append(config)

    return env_config

In [18]:
test_env_config = get_config_Test(dic, Number_Test_Days)

0
Successfully transformed into array
1
Successfully transformed into array
2
Successfully transformed into array
3
Successfully transformed into array
4
Successfully transformed into array
5
Successfully transformed into array
6
Successfully transformed into array
7
Successfully transformed into array
8
Successfully transformed into array
9
Successfully transformed into array


In [20]:
def DRL_prediction(
            model_name,
            env,
            env_instance,
            price_array,
            tech_array,
            turbulence_array,
            agent_path,
    ):
        if model_name not in MODELS:
            raise NotImplementedError("NotImplementedError")

        if model_name == "a2c":
            model_config = MODELS[model_name].A2C_DEFAULT_CONFIG.copy()
        elif model_name == "td3":
            model_config = MODELS[model_name].TD3_DEFAULT_CONFIG.copy()
        else:
            model_config = MODELS[model_name].DEFAULT_CONFIG.copy()
        model_config["env"] = env
        model_config["log_level"] = "WARN"
        model_config["env_config"] = {
            "price_array": price_array,
            "tech_array": tech_array,
            "turbulence_array": turbulence_array,
            "if_train": False,
        }

        #model_config["model"]["use_lstm"] = True
        #model_config["model"]["lstm_cell_size"] = 256
        #model_config["model"]["lstm_use_prev_action"] = True
        #model_config["model"]["lstm_use_prev_reward"] = True

        env_config = {
            "price_array": price_array,
            "tech_array": tech_array,
            "turbulence_array": turbulence_array,
            "if_train": False,
        }
        #env_instance = env(config=env_config)

        # ray.init() # Other Ray APIs will not work until `ray.init()` is called.
        if model_name == "ppo":
            trainer = MODELS[model_name].PPOTrainer(env=env, config=model_config)
        elif model_name == "a2c":
            trainer = MODELS[model_name].A2CTrainer(env=env, config=model_config)
        elif model_name == "ddpg":
            trainer = MODELS[model_name].DDPGTrainer(env=env, config=model_config)
        elif model_name == "td3":
            trainer = MODELS[model_name].TD3Trainer(env=env, config=model_config)
        elif model_name == "sac":
            trainer = MODELS[model_name].SACTrainer(env=env, config=model_config)

        print('got trainer')
        
        #try:
         #   trainer.restore(agent_path)
          #  print("Restoring from checkpoint path", agent_path)
        #except BaseException:
         #   raise ValueError("Fail to load agent!")
        
        trainer.restore(agent_path)
        print("restored agent")
        print("Restoring from checkpoint path", agent_path)

        # test on the testing env
        #state = env_instance.reset()
        episode_returns = []  # the cumulative_return / initial_account
        episode_total_assets = [env_instance.initial_total_asset]
        #done = False

        obs = env_instance.reset()
        #state = [np.zeros([256], np.float32) for _ in range(2)]
        #prev_a = 0
        #prev_r = 0.0
        done = False
        total_reward = 0.0

        print("starting the loop")

        while not done:
            #action, state, _ = trainer.compute_single_action(obs, state, prev_action=prev_a, prev_reward=prev_r)
            action = trainer.compute_single_action(obs)
            obs, reward, done = env_instance.step(action)

            prev_a = action
            prev_r = reward

            total_reward += reward

            total_asset = (
                    env_instance.amount
                    + (env_instance.price_ary[env_instance.day] * env_instance.stocks).sum()
            )
            episode_total_assets.append(total_asset)
            episode_return = total_asset / env_instance.initial_total_asset
            episode_returns.append(episode_return)
        ray.shutdown()
        print("episode return: " + str(episode_return))
        print("Test Finished!")
        return episode_total_assets

In [23]:
def DRL_prediction(
            model_name,
            env,
            env_instance,
            price_array,
            tech_array,
            turbulence_array,
            agent_path,
    ):
        if model_name not in MODELS:
            raise NotImplementedError("NotImplementedError")

        if model_name == "a2c":
            model_config = MODELS[model_name].A2C_DEFAULT_CONFIG.copy()
        elif model_name == "td3":
            model_config = MODELS[model_name].TD3_DEFAULT_CONFIG.copy()
        else:
            model_config = MODELS[model_name].DEFAULT_CONFIG.copy()
        model_config["env"] = env
        model_config["log_level"] = "WARN"
        model_config["env_config"] = {
            "price_array": price_array,
            "tech_array": tech_array,
            "turbulence_array": turbulence_array,
            "if_train": False,
        }

        #model_config["model"]["use_lstm"] = True
        #model_config["model"]["lstm_cell_size"] = 256
        #model_config["model"]["lstm_use_prev_action"] = True
        #model_config["model"]["lstm_use_prev_reward"] = True

        env_config = {
            "price_array": price_array,
            "tech_array": tech_array,
            "turbulence_array": turbulence_array,
            "if_train": False,
        }
        #env_instance = env(config=env_config)

        # ray.init() # Other Ray APIs will not work until `ray.init()` is called.
        if model_name == "ppo":
            trainer = MODELS[model_name].PPOTrainer(env=env, config=model_config)
        elif model_name == "a2c":
            trainer = MODELS[model_name].A2CTrainer(env=env, config=model_config)
        elif model_name == "ddpg":
            trainer = MODELS[model_name].DDPGTrainer(env=env, config=model_config)
        elif model_name == "td3":
            trainer = MODELS[model_name].TD3Trainer(env=env, config=model_config)
        elif model_name == "sac":
            trainer = MODELS[model_name].SACTrainer(env=env, config=model_config)

        print('got trainer')
        
        #try:
         #   trainer.restore(agent_path)
          #  print("Restoring from checkpoint path", agent_path)
        #except BaseException:
         #   raise ValueError("Fail to load agent!")
        
        trainer.restore(agent_path)
        print("restored agent")
        print("Restoring from checkpoint path", agent_path)

        state = env_instance.reset()
        episode_returns = []  # the cumulative_return / initial_account
        episode_total_assets = [env_instance.initial_total_asset]
        done = False
        while not done:
            action = trainer.compute_single_action(state)
            state, reward, done, _ = env_instance.step(action)

            total_asset = (
                    env_instance.amount
                    + (env_instance.price_ary[env_instance.day] * env_instance.stocks).sum()
            )
            episode_total_assets.append(total_asset)
            episode_return = total_asset / env_instance.initial_total_asset
            episode_returns.append(episode_return)
        ray.shutdown()
        print("episode return: " + str(episode_return))
        print("Test Finished!")
        return episode_total_assets

In [21]:
def test(
        env_config,
        env,
        capital,
        model_name,
        cwd,
        if_vix=True,
        **kwargs
):

    env_instance = env(config=env_config, initial_capital=capital)

    # load elegantrl needs state dim, action dim and net dim
    net_dimension = kwargs.get("net_dimension", 2 ** 7)
    #cwd = "./trained_" + str(model_name)

    print("price_array: ", len(env_config["price_array"]))

        # load agent
    episode_total_assets = DRL_prediction(
            model_name=model_name,
            env=env,
            env_instance = env_instance,
            price_array=env_config["price_array"],
            tech_array=env_config["tech_array"],
            turbulence_array=env_config["turbulence_array"],
            agent_path=cwd,
        )

    return episode_total_assets



In [24]:
ray.shutdown()

In [25]:
test_results = []
final_capital = [starting_capital]

for index in range(len(test_env_config)):

      print(index)

      account_value = test(env_config = test_env_config[index],
                      env=env, 
                      capital = final_capital[index], 
                      model_name=model_algorithm,
                      cwd =  checkpoint_path,
                      net_dimension = 512)
      final_capital.append(account_value[-1])
      ray.shutdown()
      test_results.append(account_value)
      

0
price_array:  481


2022-05-03 12:40:40,635	INFO services.py:1462 -- View the Ray dashboard at http://127.0.0.1:8265
(RolloutWorker pid=17964) 2022-05-03 12:40:58,234	WARNING rollout_worker.py:499 -- We've added a module for checking environments that are used in experiments. It will cause your environment to fail if your environment is not set upcorrectly. You can disable check env by setting `disable_env_checking` to True in your experiment config dictionary. You can run the environment checking module standalone by calling ray.rllib.utils.check_env(env).
(RolloutWorker pid=17964) 2022-05-03 12:40:58,234	WARNING env.py:121 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
(RolloutWorker pid=17966) 2022-05-03 12:40:58,294	WARNING rollout_worker.py:499 -- We've added a module for checking environments that are 

got trainer


2022-05-03 12:41:04,464	INFO trainable.py:535 -- Restored on 172.28.0.2 from checkpoint: /tmp/rllib_checkpoint/checkpoint_000005/checkpoint-5
2022-05-03 12:41:04,470	INFO trainable.py:543 -- Current state after restoring: {'_iteration': 5, '_timesteps_total': 180000, '_time_total': 723.6820104122162, '_episodes_total': 370}


restored agent
Restoring from checkpoint path /tmp/rllib_checkpoint/checkpoint_000005/checkpoint-5
episode return: 0.9915462524757181
Test Finished!
1
price_array:  481


...

restored agent
Restoring from checkpoint path /tmp/rllib_checkpoint/checkpoint_000005/checkpoint-5
episode return: 0.981042313079841
Test Finished!
9
price_array:  481


2022-05-03 12:48:36,934	INFO services.py:1462 -- View the Ray dashboard at http://127.0.0.1:8265
(RolloutWorker pid=31709) 2022-05-03 12:49:01,296	WARNING rollout_worker.py:499 -- We've added a module for checking environments that are used in experiments. It will cause your environment to fail if your environment is not set upcorrectly. You can disable check env by setting `disable_env_checking` to True in your experiment config dictionary. You can run the environment checking module standalone by calling ray.rllib.utils.check_env(env).
(RolloutWorker pid=31709) 2022-05-03 12:49:01,296	WARNING env.py:121 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
(RolloutWorker pid=31815) 2022-05-03 12:49:08,191	WARNING rollout_worker.py:499 -- We've added a module for checking environments that are 

got trainer


2022-05-03 12:49:27,582	INFO trainable.py:535 -- Restored on 172.28.0.2 from checkpoint: /tmp/rllib_checkpoint/checkpoint_000005/checkpoint-5
2022-05-03 12:49:27,590	INFO trainable.py:543 -- Current state after restoring: {'_iteration': 5, '_timesteps_total': 180000, '_time_total': 723.6820104122162, '_episodes_total': 370}


restored agent
Restoring from checkpoint path /tmp/rllib_checkpoint/checkpoint_000005/checkpoint-5
episode return: 0.9864126315847879
Test Finished!


In [26]:
#Function to calculate the sharpe ratio from the list of total_episode_reward
def calculate_sharpe(episode_reward:list):
  perf_data = pd.DataFrame(data=episode_reward,columns=['reward'])
  perf_data['daily_return'] = perf_data['reward'].pct_change(1)
  if perf_data['daily_return'].std() !=0:
    sharpe = (252**0.5)*perf_data['daily_return'].mean()/ \
          perf_data['daily_return'].std()
    return sharpe
  else:
    return 0

In [ ]:
df_sharpe_ratios = []
for count in range(len(test_results)):
  df_sharpe_ratios.append(calculate_sharpe(test_results[count]))
print(df_sharpe_ratios)

In [27]:
from datetime import datetime
now = datetime.now()
current_datetime = datetime.now()
print("Current date & time : ", current_datetime)
str_current_datetime = str(current_datetime)

for index in range(len(test_results)):

  df_account_test = pd.DataFrame(data=test_results[index],columns=['account_value'])
  file_name = str_current_datetime+" test "+str(index+1)+".csv"
  df_account_test.to_csv(file_name)
  files.download(file_name)

Current date & time :  2022-05-03 12:50:10.035083


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>